In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
data=pd.read_csv("../thirtysix/datasets/bayes/mushrooms.csv")

In [4]:
data.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [5]:
data.shape

(8124, 23)

In [6]:
a=np.unique(data.iloc[:,22:])

In [7]:
a

array(['d', 'g', 'l', 'm', 'p', 'u', 'w'], dtype=object)

In [8]:
li = LabelEncoder()

In [9]:
data=data.apply(li.fit_transform)

In [10]:
li.classes_

array(['d', 'g', 'l', 'm', 'p', 'u', 'w'], dtype=object)

In [11]:
data.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [12]:
data=data.values
X=data[:,1:]
y=data[:,0]

In [13]:
y.shape , X.shape

((8124,), (8124, 22))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [15]:
class NBcoustom:
    
    def fit(self,X,y):
        self.X=X
        self.y=y
        
    def prior_prop(self,label):
        total=self.y.shape[0]
        want=np.sum(self.y==label)
        
        prior =want /float(total)
        return prior
    
    def conditional_prob(self,feature_col,feature_val,label):
        
        X_filtered=self.X[self.y==label]
        numerator = np.sum(X_filtered[:, feature_col] == feature_val)
        denominator=len(X_filtered)
        return numerator/float(denominator)
    
    def predicted_point(self,X):
        classes =np.unique(self.y)
        n_feature=self.X.shape[1]
        pos_prop=[]
        
        for label in classes:
            likeyhood=1.0
            for f in range(n_feature):
                conda=self.conditional_prob(f , X[f] , label)
                #print(conda)
                likeyhood*=conda
            #print("break")
            prior=self.prior_prop(label)
            post=likeyhood*prior
            pos_prop.append(post)
            #print(pos_prop)
        return np.argmax(pos_prop)
    
    def predicted(self,X):
        res=[]
        for point in X:
            res.append(self.predicted_point(point))
        return np.array(res)
    
    def score(self, X_test, y_test):
        return (self.predicted(X_test) == y_test).mean()

In [16]:
model=NBcoustom()

In [17]:
model.fit(X_train, y_train)

In [18]:
model.predicted(X_test[:10])

array([0, 1, 1, 0, 1, 1, 1, 1, 0, 0], dtype=int64)

In [19]:
y[:10]

array([1, 0, 0, 1, 0, 0, 0, 0, 1, 0])

In [20]:
model.score(X_test,y_test)

0.9973890339425587

In [21]:
model.predicted_point(X_test[1])

1

In [22]:
model.conditional_prob(21,2,1)

0.15958668197474168

In [24]:
a=X[0]

In [29]:
a[0]

5